## Librerias

In [ ]:
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options

In [ ]:
### Evitar publicidad
###https://medium.com/@rosolko/block-an-advertisements-for-application-under-tests-52ea468ec288

## TELEGRAFO

In [ ]:
# Ruta del perfil de Firefox
profile_path = r'C:\Users\Alexis\AppData\Roaming\Mozilla\Firefox\Profiles\wl9lgny0.automation'

# Ruta de geckodriver
geckodriver_path = r'C:\Users\Alexis\Downloads\geckodriver-v0.34.0-win32\geckodriver.exe'

# Configuración de opciones de Firefox
options = Options()
options.add_argument('--profile')
options.add_argument(profile_path)
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'

# Inicializar el driver
driver = webdriver.Firefox(options=options)
driver.get("https://www.primicias.ec/lo-ultimo/")

data_list      = []
noticias_count = 0
visited_links  = []

fecha_actual = datetime.now().strftime("%Y-%m-%d")
try:
    while True:
        time.sleep(2)
        articles = driver.find_elements(By.XPATH, '//div[@class="lista__articulos"]/article')

        for article in articles:
            try:
                driver.implicitly_wait(1)
                article_link = article.find_element(By.XPATH, './/a').get_attribute("href")

                if article_link not in visited_links and "noticias" in article_link:
                    driver.implicitly_wait(1)
                    driver.execute_script("window.open('');")
                    driver.switch_to.window(driver.window_handles[1])
                    driver.get(article_link)
                    time.sleep(1)

                    # Data
                    try:
                        titulo_seccion = driver.find_element(By.XPATH, '//div[@class="titulo__seccion"]').text
                    except Exception:
                        titulo_seccion = ''

                    try:
                        noticia_titulo = driver.find_element(By.XPATH, '//h1[@class="noticia__titulo"]').text
                    except Exception:
                        try:
                            noticia_titulo = driver.find_element(By.XPATH, '//div[@class="noticia__titulo"]').text
                        except Exception:
                            noticia_titulo = ''

                    try:
                        noticia_entradilla = driver.find_element(By.XPATH, '//h3[@class="noticia__entradilla"]').text
                    except Exception:
                        noticia_entradilla = ''

                    try:
                        noticia_fecha = driver.find_element(By.XPATH, '//*[@id="post-wrapper"]/div/div[4]/div[1]/div[1]/div/div[2]/p[2]').text
                    except Exception:
                        noticia_fecha = ''

                    textos_enlaces = []
                    try:
                        header_tags_div = driver.find_element(By.XPATH, '//div[@class="header__tags desktop"]')
                        lista_items = header_tags_div.find_elements(By.XPATH, './ul/li')

                        for item in lista_items:
                            texto_enlace = item.find_element(By.XPATH, './a').text
                            textos_enlaces.append(texto_enlace)
                    except Exception:
                        textos_enlaces = []

                    textos_parrafos = []
                    try:
                        entry_content = driver.find_element(By.XPATH, '//div[@class="entry-content"]')
                        parrafos = entry_content.find_elements(By.TAG_NAME, 'p')
                        textos_parrafos = [parrafo.text for parrafo in parrafos]
                    except:
                        textos_parrafos = []

                    data_list.append({
                        'Titulo_Seccion': titulo_seccion,
                        'Noticia_Titulo': noticia_titulo,
                        'Noticia_Entradilla': noticia_entradilla,
                        'Header_Tags': textos_enlaces,
                        'Contenido_Parrafos': textos_parrafos,
                        'Link': article_link,
                        'Fecha': noticia_fecha
                    })

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                    noticias_count += 1
                    visited_links.append(article_link)

                    if noticias_count % 10 == 0:
                        df = pd.DataFrame(data_list)
                        df.to_csv(f'noticias_primicias_{noticias_count}_{fecha_actual}.csv', index=False)
                        print(f'noticias_primicias_{noticias_count}.csv saved')
            except Exception:
                continue
        try:
            driver.implicitly_wait(1)
            btn_cargar_mas = driver.find_element(By.XPATH, '//*[@id="btn__cargar_mas"]')
            btn_cargar_mas.click()
        except Exception:
            break
        
finally:
    df = pd.DataFrame(data_list)
    df.to_csv(f'noticias_primicias_final_{fecha_actual}.csv', index=False)
    print(f'noticias_primicias_final.csv saved')
    driver.quit()

## PRIMICIAS

### Enlaces

In [ ]:
# Ruta del perfil de Firefox
profile_path = r'C:\Users\Alexis\AppData\Roaming\Mozilla\Firefox\Profiles\wl9lgny0.automation'

# Ruta de geckodriver
geckodriver_path = r'C:\Users\Alexis\Downloads\geckodriver-v0.34.0-win32\geckodriver.exe'

# Configuración de opciones de Firefox
options = Options()
options.add_argument('--profile')
options.add_argument(profile_path)
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'

# Inicializar el driver
driver = webdriver.Firefox(options=options)
driver.get("https://www.eltelegrafo.com.ec")

data_list        = []
enlaces_noticias = []
noticias_count   = 0
fecha_actual     = datetime.now().strftime("%Y-%m-%d")

def guardar_enlaces(enlaces, count):
    """Guardar enlaces en un archivo CSV."""
    df = pd.DataFrame(enlaces, columns=['Enlace'])
    file_name = f'enlaces_noticias_{count}_{fecha_actual}.csv'
    df.to_csv(file_name, index=False)
    print(f'{file_name} guardado.')

try:
    while True:
        try:
            time.sleep(2)
            next_button = driver.find_element(By.XPATH, '//div[@class="amazingcarousel-next"]')
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(2)
            
            nuevos_enlaces = driver.find_elements(By.CSS_SELECTOR, '.amazingcarousel-item-container a')
            for enlace in nuevos_enlaces:
                href = enlace.get_attribute('href')
                if href not in enlaces_noticias:
                    enlaces_noticias.append(href)
                    noticias_count += 1
                    #print(f'Enlace agregado: {href}')
                    
                    if noticias_count % 10 == 0:
                        guardar_enlaces(enlaces_noticias, noticias_count)

        except Exception as e:
            print(f"Error al procesar: {e}")
            break
finally:
    if noticias_count % 10 != 0:
        guardar_enlaces(enlaces_noticias, noticias_count)
    
    driver.quit()

### Noticias

In [ ]:
# Ruta del perfil de Firefox
profile_path = r'C:\Users\Alexis\AppData\Roaming\Mozilla\Firefox\Profiles\wl9lgny0.automation'

# Ruta de geckodriver
geckodriver_path = r'C:\Users\Alexis\Downloads\geckodriver-v0.34.0-win32\geckodriver.exe'

# Configuración de opciones de Firefox
options = Options()
options.add_argument('--profile')
options.add_argument(profile_path)
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'

# Inicializar el driver
driver = webdriver.Firefox(options=options)

# Leer el archivo de enlaces
df = pd.read_csv('enlaces_noticias_20_2024-06-19.csv') # Nombre del archivo
enlaces_noticias = df['Enlace'].tolist()

data_list      = []
noticias_count = 0
fecha_actual = datetime.now().strftime("%Y-%m-%d")

try:
    for enlace in enlaces_noticias:
        driver.get(enlace)
        time.sleep(1)

        try:
            article_title_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//h1[contains(@class, "story-heading")]/span'))
            )
            article_title = article_title_element.text.strip()
        except Exception as e:
            print(f"Error retrieving article title: {str(e)}")
            article_title = ''

        try:
            article_category_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "story-category")]/a'))
            )
            article_category = article_category_element.text.strip()
        except Exception as e:
            print(f"Error retrieving article category: {str(e)}")
            article_category = ''

        try:
            article_publish_date_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//span[contains(@class, "story-publishup")]'))
            )
            article_publish_date = article_publish_date_element.text.strip()
        except Exception as e:
            print(f"Error retrieving article publish date: {str(e)}")
            article_publish_date = ''

        try:
            article_content_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "itemFullText")]'))
            )
            article_content = article_content_element.text.strip()
        except Exception as e:
            print(f"Error retrieving article content: {str(e)}")
            article_content = ''

        data_list.append({
            'Title': article_title,
            'Category': article_category,
            'Publish Date': article_publish_date,
            'Content': article_content,
            'Link': enlace
        })

        noticias_count += 1

        if noticias_count % 10 == 0:
            df = pd.DataFrame(data_list)
            df.to_csv(f'dataset_telegrafo_{noticias_count}_{fecha_actual}.csv', index=False)
            print(f'dataset_telegrafo_{noticias_count}_{fecha_actual}.csv guardado')

finally:
    df = pd.DataFrame(data_list)
    df.to_csv(f'dataset_final_telegrafo_{fecha_actual}.csv', index=False)
    print(f'dataset_final_telegrafo_{fecha_actual}.csv guardado')

    # Cerrar el navegador
    driver.quit()